In [2]:
import pandas as pd
import json


In [3]:
df = pd.read_csv('annotations.csv')

In [ ]:
df.label.unique()

In [ ]:
df = df[df["annotation_type"] == "KeyPoint"][ df["label"] == "ny_marklinje"]

In [ ]:
df

In [7]:
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib import patches
import numpy as np

def visualize_image_with_bounding_boxes_colored(df):
    """
    Visualizes an image with bounding boxes, each colored according to the rectanglelabels.
    Adds labels and colors to the legend.
    
    Parameters:
    - image_path: Path to the image file.
    - bounding_boxes: A list of bounding boxes, where each bounding box is represented as a dictionary
      with keys 'x', 'y', 'width', 'height', and optionally 'label'. Coordinates are normalized to [0, 1].
    """
    # Open the image file
    img = Image.open(df["image"])
    fig, ax = plt.subplots(1)
    fig.set_size_inches(10, 10)
    ax.imshow(img)
    
    # Image dimensions
    img_width, img_height = img.size
    
    # Label to color mapping
    label_color_map = {
        'riktning_text': 'r',
        'another_label': 'g',
          # Example additional label
        # Add more labels and colors as needed
    }
    

    # Denormalize coordinates
    x = df['x'] * img_width //100
    y = df['y'] * img_height //100
    
    # Get color for the label
    label = df['label']
    color = label_color_map.get(label, 'b')  # Default to blue if label not in map
    
    # Create a Rectangle patch
    rect = patches.Rectangle((x, y), 2, 2, linewidth=2, edgecolor=color, facecolor=color, label=label)
    
    # Add the patch to the Axes
    ax.add_patch(rect)
    
    # Create legend from unique labels
    handles, labels = ax.get_legend_handles_labels()
    by_label = dict(zip(labels, handles))  # Removing duplicates
    ax.legend(by_label.values(), by_label.keys())
    
    return plt

In [ ]:
df.iloc[0]

In [ ]:
drawings = df.groupby('task_id').first().reset_index()
drawings

In [13]:
#data loader for the dataset
import torch
from PIL import Image
from torchvision.transforms import functional as F
import os
import cv2

class KBABygglovDataset(torch.utils.data.Dataset):
    def __init__(self, drawings, df,scale = 1):
        self.df = df
        self.drawings = drawings
        self.transforms = None
        self.scale = scale
        #self.task_ids = df["task_id"].unique()
    
    def __len__(self):
        return len(self.drawings)
    
    def __getitem__(self, idx):
        task_id = self.drawings.iloc[idx]["task_id"]
        rows = self.df[self.df["task_id"] == task_id]
        image = cv2.imread(rows.iloc[0]["image"])
        #contour = find_bounding_box(image)
        
        
        # Normalize bounding box coordinates
        width, height = rows.iloc[0]["original_width"], rows.iloc[0]["original_height"]
        target = {}
        points = []
        labels = []
        boxes = [[0,0, image.shape[0], image.shape[1]]]

        for i, row in rows.iterrows():
            x1 = row['x'] * row["original_width"] //100 
            y1 = row['y'] * row["original_height"] //100
            points.append([x1, y1, 1])
            labels.append(1)
            if len(points) == 10:
                break
        while len(points) != 10:
            points.append([0,0,0])
        target["keypoints"] = torch.as_tensor(points, 
                                dtype = torch.float32)
        target["boxes"] = torch.as_tensor(boxes, 
                                dtype = torch.float32)
        target["labels"]=torch.as_tensor([0],
                        dtype = torch.int64)
        target["image_id"] = torch.as_tensor([task_id])
        #scale the image and target to half size
        image = cv2.resize(image, (image.shape[1]// self.scale,  image.shape[0]// self.scale))

        target["boxes"] = target["boxes"]/ self.scale
        target["keypoints"] = target["keypoints"]/ self.scale
        
        image = F.to_tensor(image)
        #cropped_image, target = crop_to_contour(image, target, contour)
        return image, target




dataset = KBABygglovDataset(drawings.iloc[:4* len(drawings)//5], df)
test_dataset = KBABygglovDataset(drawings.iloc[4* len(drawings)//5:], df)

In [ ]:
len(dataset)

In [ ]:
len(test_dataset)

In [ ]:
for data in dataset:
    image, boxes = data
    image.to("cpu")
    print(boxes)
    #print(image.shape)
    break

In [17]:
# Collate image-target pairs into a tuple.
def collate_fn(batch):
    return tuple(zip(*batch))
# Create the DataLoaders from the Datasets. 
train_dl = torch.utils.data.DataLoader(dataset, 
                                 batch_size = 4, 
                                 shuffle = True, 
                        collate_fn = collate_fn)
'''val_dl = torch.utils.data.DataLoader(val_ds, 
                             batch_size = 4, 
                            shuffle = False, 
                    collate_fn = collate_fn)'''
test_dl = torch.utils.data.DataLoader(test_dataset, 
                               batch_size = 1, 
                              shuffle = False, 
                      collate_fn = collate_fn)

In [ ]:
import os, json, cv2, numpy as np, matplotlib.pyplot as plt

import torch
from torch.utils.data import Dataset, DataLoader

import torchvision
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.transforms import functional as F

import albumentations as A # Library for augmentations

model = torchvision.models.detection.keypointrcnn_resnet50_fpn(pretrained=False,
                                                                   pretrained_backbone=False,
                                                                   num_keypoints=10,
                                                                   num_classes = 1, # Background is the first class, object is the second class
                                                              )
  
        


In [22]:
def unbatch(batch, device):
    """
    Unbatches a batch of data from the Dataloader.
    Inputs
        batch: tuple
            Tuple containing a batch from the Dataloader.
        device: str
            Indicates which device (CPU/GPU) to use.
    Returns
        X: list
            List of images.
        y: list
            List of dictionaries.
    """
    X, y = batch
    X = [x.to(device) for x in X]
    y = [{k: v.to(device) for k, v in t.items()} for t in y]
    return X, y
def train_batch(batch, model, optimizer, device):
    """
    Uses back propagation to train a model.
    Inputs
        batch: tuple
            Tuple containing a batch from the Dataloader.
        model: torch model
        optimizer: torch optimizer
        device: str
            Indicates which device (CPU/GPU) to use.
    Returns
        loss: float
            Sum of the batch losses.
        losses: dict
            Dictionary containing the individual losses.
    """
    model.train()
    X, y = unbatch(batch, device = device)
    optimizer.zero_grad()
    losses = model(X, y)
    loss = sum(loss for loss in losses.values())
    loss.backward()
    optimizer.step()
    return loss, losses
@torch.no_grad()
def validate_batch(batch, model, optimizer, device):
    """
    Evaluates a model's loss value using validation data.
    Inputs
        batch: tuple
            Tuple containing a batch from the Dataloader.
        model: torch model
        optimizer: torch optimizer
        device: str
            Indicates which device (CPU/GPU) to use.
    Returns
        loss: float
            Sum of the batch losses.
        losses: dict
            Dictionary containing the individual losses.
    """
    model.train()
    X, y = unbatch(batch, device = device)
    optimizer.zero_grad()

    losses = model(X, y)
    loss = sum(loss for loss in losses.values())
    return loss, losses

In [ ]:

import torch
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torch.utils.data import DataLoader
from torchvision.transforms import functional as F
from torch_snippets import Report


num_epochs = 20
# Assuming 'dataset' is an instance of 'KBABygglovDataset' and 'data_loader' is an instance of 'DataLoader'
# Also assuming 'device' is defined (e.g., cuda or cpu)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, 
                        lr = 0.005, 
                    momentum = 0.9, 
             weight_decay = 0.0005)


log = Report(num_epochs)
# FasterRCNN loss names.
keys = ["loss_classifier", 
            "loss_box_reg", 
        "loss_objectness", 
        "loss_rpn_box_reg"]
model.to(device)
for epoch in range(num_epochs):
    N = len(train_dl)
    for ix, batch in enumerate(train_dl):
        loss, losses = train_batch(batch, model, 
                                optimizer, device)
        # Record the current train loss.
        pos = epoch + (ix + 1) / N
        log.record(pos = pos, trn_loss = loss.item(), 
                    end = "\r")
    if test_dl is not None:
        N = len(test_dl)
        for ix, batch in enumerate(test_dl):
            loss, losses = validate_batch(batch, model, 
                                        optimizer, device)
            
            # Record the current validation loss.
            pos = epoch + (ix + 1) / N
            log.record(pos = pos, val_loss = loss.item(), 
                        end = "\r")
log.report_avgs(epoch + 1)
log

In [15]:
torch.save(model, 'marklinje.model')

In [18]:
model = torch.load('marklinje.model',  map_location=torch.device('cpu')).eval()

In [23]:
@torch.no_grad()
def predict_batch(batch, model, device):
    model.to(device)
    model.eval()
    X, _ = unbatch(batch, device = device)
    predictions = model(X)
    return [x.cpu() for x in X], predictions
def predict(model, data_loader, device = "cpu"):
    images = []
    predictions = []
    for i, batch in enumerate(data_loader):
        print(i)
        X, p = predict_batch(batch, model, device)
        images.append(X)
        predictions.append(p)
    
    return images, predictions

In [ ]:
len(train_dl)

In [ ]:
images, predictions = predict(model, train_dl, device = "cuda")

In [ ]:
predictions